In [4]:
from diffusers import FlaxStableDiffusionPipeline
import jax
import jax.numpy as jnp

with jax.default_device(jax.devices("cpu")[0]):
    pipeline, params = FlaxStableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base", dtype=jnp.float16,
        safety_checker=None,
        from_pt=True
    )

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/daniel/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1-base/snapshots/5ede9e4bf3e3fd1cb0ef2f7a3fff13ee514fdf06/text_encoder were not used when initializing FlaxCLIPTextModel: {('text_model', 'embeddings', 'position_ids')}
- This IS expected if you are initializing FlaxCLIPTextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxCLIPTextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_flax_stable_diffusion.FlaxStableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion l

In [13]:
import flax.jax_utils

prompt = "A cinematic film still of Morgan Freeman starring as Jimi Hendrix, portrait, 40mm lens, shallow depth of field, close up, split lighting, cinematic"
prompt = [prompt] * jax.device_count()
prompt_ids = pipeline.prepare_inputs(prompt)
prompt_ids.shape

p_params = flax.jax_utils.replicate(params)

images = pipeline(prompt_ids, p_params, jax.random.split(jax.random.key(42),1), jit=True).images
images = images.reshape((images.shape[0] * images.shape[1],) + images.shape[-3:])
images = pipeline.numpy_to_pil(images)

2024-11-24 17:10:02.775615: W external/xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 193.00MiB (rounded to 202375168)requested by op 
2024-11-24 17:10:02.776625: W external/xla/xla/tsl/framework/bfc_allocator.cc:508] *********************************************************************************************__*****
E1124 17:10:02.776657  494009 pjrt_stream_executor_client.cc:3085] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 202375168 bytes.


XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 202375168 bytes.

In [ ]:
import flax.jax_utils

